In [1]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import torch
from os import listdir
from os.path import isfile, join
from sklearn import preprocessing
import torchvision.transforms as transforms
import torchvision.models as models
from torch import nn
import torch.utils.data

In [2]:
#Get link to directory
path_to_train_info = os.path.join(os.getenv('DATASETS'), 'ArtRecognision\\train_info.csv')
path_to_train_set = os.path.join(os.getenv('DATASETS'), 'ArtRecognision\\train_2\\')
path_to_transformed_train_set = os.path.join(os.getenv('DATASETS'), 'ArtRecognision\\transformed_images\\')

In [3]:
#Load datasets
train_set = pd.read_csv(path_to_train_info)
train_set

,filename,artist,title,style,genre,date
0,102257.jpg,5b39c876740bfc1cfaf544721c43cac3,Uriel,Color Field Painting,abstract,1955.0
1,75232.jpg,5b39c876740bfc1cfaf544721c43cac3,Vir Heroicus Sublimis,Color Field Painting,abstract,1950.0
2,29855.jpg,96e5bc98488ed589b9bf17ad9fd09371,Night March of a Hundred Demons (left half),Yamato-e,mythological painting,NaN
3,62252.jpg,5b39c876740bfc1cfaf544721c43cac3,"Who’s Afraid of Red, Yellow and Blue II",Color Field Painting,abstract,NaN
4,63861.jpg,5b39c876740bfc1cfaf544721c43cac3,Black Fire I,Color Field Painting,abstract,1963.0
...,...,...,...,...,...,...
79428,23862.jpg,67959e4e5df05b3d9db7c97fd9a0b0f6,Number 547,Abstract Expressionism,NaN,1954.0
79429,25525.jpg,2d72f2000c42051e7c350a39bdce9bc1,Number 13A (Arabesque),Action painting,abstract,1948.0
79430,47038.jpg,f920b951670e3d8dc3c759f12ced7a3e,St. Francis of Assisi,Baroque,religious painting,NaN
79431,9021.jpg,e4183fd3d19c2bca8b7c56d19af92252,Thebe's Revenge,NaN,marina,1982.0


In [4]:
#Get list of images
onlyfiles = [f for f in listdir(path_to_train_set) if isfile(join(path_to_train_set, f))]

In [5]:
#Leave data only for the downloaded images
filtered_train_set = train_set[train_set.filename.isin(onlyfiles)]
final_train_dataset = filtered_train_set[['filename', 'style', 'genre']].dropna()

In [18]:
def lowest_size(name_list, directory_path):
    min_list = []
    for name in name_list:
        img=cv2.imread(directory_path + name)
        height, width, _ = img.shape
        if height < width: min_list.append(height) 
        else: min_size = min_list.append(width) 
    
    min_size = min(min_list)
    return min_size

In [22]:
def lowest_size(name_list, directory_path):
    min_list = name_list.tolist()
    img = cv2.imread(directory_path + min_list[0])
    height, width, _ = img.shape
    min_value = min(height, width)
    for name in name_list:
        img=cv2.imread(directory_path + name)
        height, width, _ = img.shape
        m1 = min(height, width)
        if m1 < min_value: min_value = m1

    return min_value

In [ ]:
aws = lowest_size(final_train_dataset['filename'], path_to_train_set)

In [20]:
my_min_function(somelist)

приваивание минимума 10
приваивание минимума 35


35

In [6]:
#fuction to crop images with square
def crop_image(image, crop_size):
    height, width, _ = image.shape
    cropped_img = image[(height-crop_size)//2 : (height+crop_size)//2, (width-crop_size)//2 : (width+crop_size)//2].copy()
    return cropped_img

In [ ]:
#image pre-processing
crop_size = lowest_size(final_train_dataset['filename'], path_to_train_set)
img_size = 224

for name in final_train_dataset['filename']:
    try:
        img=cv2.imread(path_to_train_set + name)
        c_image = crop_image(img, crop_size)
        r_image=cv2.resize(c_image,(img_size,img_size),interpolation=cv2.INTER_AREA)
        t_image = torch.Tensor(r_image)
        new_name = name.split(".", 1)[0] + ".pt"
        torch.save(t_image, path_to_transformed_train_set + new_name)

    except:
        continue        

tensor([[[198., 214., 221.],
         [202., 216., 223.],
         [200., 216., 223.],
         ...,
         [124., 139., 161.],
         [107., 121., 142.],
         [100., 114., 134.]],

        [[199., 216., 223.],
         [202., 215., 223.],
         [200., 216., 223.],
         ...,
         [151., 172., 198.],
         [144., 167., 193.],
         [135., 156., 183.]],

        [[198., 213., 220.],
         [201., 217., 224.],
         [198., 216., 223.],
         ...,
         [150., 174., 200.],
         [148., 174., 200.],
         [150., 176., 203.]],

        ...,

        [[179., 200., 210.],
         [178., 197., 211.],
         [184., 205., 220.],
         ...,
         [ 77.,  93., 105.],
         [ 81.,  96., 110.],
         [ 86.,  95., 113.]],

        [[184., 205., 215.],
         [180., 199., 214.],
         [183., 203., 218.],
         ...,
         [105., 121., 133.],
         [102., 118., 131.],
         [110., 123., 137.]],

        [[182., 202., 212.],
       

tensor([[[129., 168., 217.],
         [131., 171., 220.],
         [132., 175., 222.],
         ...,
         [172., 151., 133.],
         [171., 145., 128.],
         [168., 144., 130.]],

        [[128., 168., 214.],
         [134., 175., 220.],
         [134., 177., 223.],
         ...,
         [172., 152., 133.],
         [163., 142., 127.],
         [153., 135., 124.]],

        [[124., 164., 212.],
         [132., 172., 219.],
         [139., 179., 223.],
         ...,
         [163., 148., 132.],
         [156., 140., 128.],
         [141., 136., 130.]],

        ...,

        [[ 56.,  86., 106.],
         [ 56.,  90., 128.],
         [ 69., 105., 154.],
         ...,
         [ 29.,  83., 149.],
         [ 37., 101., 166.],
         [ 41.,  98., 155.]],

        [[ 50.,  77., 101.],
         [ 57.,  90., 127.],
         [ 66., 102., 149.],
         ...,
         [ 35.,  83., 150.],
         [ 38.,  95., 154.],
         [ 45.,  89., 140.]],

        [[ 57.,  85., 113.],
       

tensor([[[146.,  83.,  89.],
         [172., 115., 107.],
         [184., 144., 126.],
         ...,
         [157., 127., 133.],
         [150., 136., 139.],
         [152., 135., 141.]],

        [[164., 107.,  99.],
         [179., 124., 117.],
         [178., 137., 133.],
         ...,
         [160., 130., 134.],
         [152., 132., 144.],
         [156., 138., 141.]],

        [[173., 125., 109.],
         [174., 120., 112.],
         [166., 114., 111.],
         ...,
         [161., 136., 139.],
         [160., 135., 139.],
         [158., 131., 131.]],

        ...,

        [[179., 141., 132.],
         [177., 141., 124.],
         [175., 125., 126.],
         ...,
         [178., 118., 104.],
         [182., 125., 112.],
         [193., 139., 125.]],

        [[178., 129., 123.],
         [182., 134., 126.],
         [180., 128., 127.],
         ...,
         [181., 123., 106.],
         [175., 115., 100.],
         [186., 126., 114.]],

        [[182., 136., 124.],
       

tensor([[[106.,  93.,  96.],
         [111., 109.,  90.],
         [118., 112., 108.],
         ...,
         [ 76.,  78.,  88.],
         [ 69.,  66.,  74.],
         [ 69.,  68.,  75.]],

        [[101., 102.,  97.],
         [108., 107., 100.],
         [ 97.,  96.,  88.],
         ...,
         [ 64.,  64.,  75.],
         [ 69.,  64.,  69.],
         [ 67.,  67.,  70.]],

        [[119., 105.,  94.],
         [107.,  98.,  89.],
         [ 92.,  85.,  85.],
         ...,
         [ 68.,  68.,  75.],
         [ 74.,  72.,  79.],
         [ 71.,  73.,  81.]],

        ...,

        [[143., 154., 164.],
         [142., 159., 169.],
         [130., 145., 156.],
         ...,
         [117., 118., 133.],
         [119., 129., 136.],
         [100., 100., 111.]],

        [[153., 158., 167.],
         [134., 155., 166.],
         [129., 153., 164.],
         ...,
         [114., 118., 130.],
         [103., 117., 126.],
         [104., 112., 123.]],

        [[136., 151., 162.],
       

tensor([[[140., 180., 212.],
         [147., 190., 221.],
         [145., 185., 217.],
         ...,
         [ 39.,  45.,  64.],
         [ 35.,  50.,  59.],
         [ 91., 119., 124.]],

        [[140., 176., 209.],
         [147., 187., 219.],
         [145., 185., 217.],
         ...,
         [ 43.,  52.,  74.],
         [ 24.,  37.,  48.],
         [ 94., 111., 117.]],

        [[141., 176., 208.],
         [148., 188., 220.],
         [149., 188., 220.],
         ...,
         [ 15.,  30.,  51.],
         [ 36.,  48.,  60.],
         [ 34.,  41.,  48.]],

        ...,

        [[143., 189., 220.],
         [147., 193., 224.],
         [148., 195., 226.],
         ...,
         [152., 196., 227.],
         [149., 193., 224.],
         [152., 196., 227.]],

        [[141., 191., 221.],
         [147., 199., 229.],
         [143., 197., 226.],
         ...,
         [156., 200., 231.],
         [154., 198., 229.],
         [147., 191., 222.]],

        [[148., 193., 224.],
       

...

tensor([[[171., 177., 179.],
         [184., 191., 192.],
         [170., 180., 183.],
         ...,
         [200., 206., 210.],
         [209., 214., 217.],
         [211., 216., 219.]],

        [[171., 180., 184.],
         [175., 179., 183.],
         [175., 181., 184.],
         ...,
         [196., 208., 217.],
         [205., 214., 222.],
         [199., 206., 211.]],

        [[158., 167., 171.],
         [166., 170., 174.],
         [176., 181., 184.],
         ...,
         [177., 195., 202.],
         [195., 210., 216.],
         [191., 199., 203.]],

        ...,

        [[209., 213., 218.],
         [209., 213., 218.],
         [209., 213., 218.],
         ...,
         [151., 138., 132.],
         [165., 149., 138.],
         [157., 142., 134.]],

        [[209., 213., 218.],
         [209., 213., 218.],
         [209., 213., 218.],
         ...,
         [150., 138., 132.],
         [157., 142., 130.],
         [154., 137., 126.]],

        [[208., 212., 217.],
       

tensor([[[ 90.,  89.,  13.],
         [ 97.,  96.,  17.],
         [ 97.,  96.,  13.],
         ...,
         [170., 175., 110.],
         [183., 181., 105.],
         [190., 183., 118.]],

        [[ 93.,  92.,  12.],
         [ 96.,  95.,  14.],
         [ 94.,  96.,  14.],
         ...,
         [170., 171., 105.],
         [187., 181., 111.],
         [176., 173., 110.]],

        [[ 87.,  90.,  11.],
         [ 95.,  97.,  17.],
         [ 98.,  97.,  16.],
         ...,
         [160., 165., 103.],
         [161., 166., 108.],
         [152., 166., 107.]],

        ...,

        [[ 16.,  34.,  17.],
         [  6.,  17.,   5.],
         [  1.,  12.,   3.],
         ...,
         [ 83., 113.,  98.],
         [ 90., 109.,  80.],
         [102., 109.,  68.]],

        [[ 16.,  33.,  17.],
         [  5.,  16.,   4.],
         [  1.,  11.,   3.],
         ...,
         [ 81., 113.,  96.],
         [100., 120.,  90.],
         [107., 115.,  74.]],

        [[ 22.,  38.,  19.],
       

tensor([[[ 65.,  67.,  63.],
         [ 66.,  64.,  60.],
         [ 72.,  72.,  67.],
         ...,
         [ 23.,  55.,  85.],
         [ 30.,  57.,  88.],
         [ 29.,  60., 101.]],

        [[ 64.,  67.,  64.],
         [ 69.,  69.,  63.],
         [ 81.,  80.,  68.],
         ...,
         [ 19.,  47.,  69.],
         [ 21.,  46.,  69.],
         [ 29.,  59.,  97.]],

        [[ 68.,  72.,  66.],
         [ 71.,  73.,  63.],
         [ 83.,  81.,  71.],
         ...,
         [ 10.,  29.,  48.],
         [ 16.,  41.,  70.],
         [ 30.,  64., 100.]],

        ...,

        [[  4.,  11.,  30.],
         [  3.,   9.,  28.],
         [  4.,  12.,  30.],
         ...,
         [ 40.,  68.,  63.],
         [ 34.,  60.,  53.],
         [ 39.,  62.,  62.]],

        [[  5.,  12.,  31.],
         [  4.,  10.,  29.],
         [  4.,  11.,  34.],
         ...,
         [ 45.,  68.,  64.],
         [ 36.,  64.,  59.],
         [ 36.,  65.,  59.]],

        [[  0.,   7.,  26.],
       

tensor([[[189., 205., 213.],
         [187., 206., 214.],
         [193., 208., 216.],
         ...,
         [186., 204., 213.],
         [183., 201., 210.],
         [183., 202., 212.]],

        [[192., 209., 217.],
         [189., 205., 214.],
         [185., 205., 213.],
         ...,
         [182., 200., 210.],
         [174., 198., 207.],
         [178., 200., 209.]],

        [[183., 203., 211.],
         [176., 199., 209.],
         [184., 202., 211.],
         ...,
         [174., 195., 205.],
         [173., 196., 205.],
         [185., 204., 213.]],

        ...,

        [[178., 192., 200.],
         [153., 174., 182.],
         [148., 169., 177.],
         ...,
         [125., 143., 147.],
         [ 89., 103., 104.],
         [ 88., 103., 107.]],

        [[120., 139., 146.],
         [167., 186., 194.],
         [175., 195., 203.],
         ...,
         [ 73.,  86.,  89.],
         [ 69.,  81.,  81.],
         [129., 143., 147.]],

        [[177., 191., 199.],
       

tensor([[[ 77.,  31.,  39.],
         [ 78.,  31.,  39.],
         [ 79.,  32.,  40.],
         ...,
         [ 77.,  30.,  38.],
         [ 79.,  32.,  40.],
         [ 79.,  32.,  40.]],

        [[ 78.,  32.,  38.],
         [ 78.,  31.,  39.],
         [ 79.,  32.,  40.],
         ...,
         [ 78.,  31.,  39.],
         [ 79.,  32.,  40.],
         [ 79.,  32.,  40.]],

        [[ 83.,  35.,  37.],
         [ 79.,  31.,  41.],
         [ 79.,  32.,  39.],
         ...,
         [ 78.,  31.,  39.],
         [ 79.,  32.,  40.],
         [ 78.,  31.,  39.]],

        ...,

        [[114.,  72.,  88.],
         [114.,  72.,  89.],
         [116.,  73.,  90.],
         ...,
         [113.,  71.,  88.],
         [114.,  72.,  89.],
         [115.,  73.,  90.]],

        [[115.,  70.,  87.],
         [114.,  70.,  87.],
         [115.,  72.,  89.],
         ...,
         [112.,  70.,  87.],
         [117.,  73.,  90.],
         [113.,  69.,  86.]],

        [[116.,  71.,  88.],
       

In [47]:
discriminator = models.resnet18(pretrained=True)

In [ ]:
discriminator.fc = nn.Sequential(nn.Linear(1000, 512),
                                 nn.Tanh(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 1))

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        n_features = 100

        def block(in_channels, out_channels, kernel_size, stride, padding)
            layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)]
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append([nn.ConvTranspose2d(in_channels, out_channels, 1, 1, 1)])
            layers.append([nn.ConvTranspose2d(in_channels, out_channels, 1, 1, 1)])
            layers.append([nn.ConvTranspose2d(in_channels, out_channels, 1, 1, 1)])
            return layers     
                
        self.linear1= nn.Linear(n_features, 256)
        self.linear2= nn.Linear(256, 6272)
        
        self.deconv1 = nn.Sequential(*block(some_input, 128, 4, 2, 1))      
        self.deconv2 = nn.Sequential(*block(128, 128, 4, 2, 1))        
        self.deconv3 = nn.Sequential(*block(256, 128, 4, 2, 1))
        self.deconv4 = nn.Sequential(*block(384, 128, 4, 2, 1))
        self.deconv5 = nn.Sequential(*block(512, 128, 4, 2, 1))
        self.deconv6 = nn.Sequential(*block(128, 64, 1, 1, 1))
        self.deconv7 = nn.Sequential(*block(64, 32, 1, 1, 1))
        self.deconv8 = nn.Sequential(*block(32, 3, 1, 1, 1))
        
        
        def forward(self, n_features = 100):
            
            linear1 = F.LeakyReLU(self.linear1(n_features))
            linear2 = F.LeakyReLU(self.linear2(linear1))
            hidden1 = linear2.view(-1, 128*7*7)
            deconv1 = F.LeakyReLU(self.deconv1(hidden1))
            deconv2 = F.LeakyReLU(self.deconv2(deconv1))
            c2_dense = F.LeakyReLU(torch.cat([deconv1, deconv2], 1))
            deconv3 = F.LeakyReLU(self.deconv3(c2_dense))
            c3_dense = F.LeakyReLU(torch.cat([deconv1, deconv2, deconv3], 1))                  
            deconv4 = F.LeakyReLU(self.deconv4(c3_dense))           
            c4_dense = F.LeakyReLU(torch.cat([deconv1, deconv2, deconv3, deconv4], 1))   
            deconv5 = F.LeakyReLU(self.deconv5(c4_dense))
            c5_dense = F.LeakyReLU(torch.cat([deconv1, deconv2, deconv3, deconv4, deconv5], 1))
            deconv6 = F.LeakyReLU(self.deconv6(c5_dense))
            deconv7 = F.LeakyReLU(self.deconv7(deconv6))
            deconv8 = F.tanh(self.deconv8(deconv7))
            
            return deconv8

In [ ]:
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
generator = Generator().to(device)
discriminator = discriminator.to(device)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.002, amsgrad=False)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.002, amsgrad=False)

In [ ]:
train_images = torch.load(path_to_transformed_train_set)
train_dataset = torch.utils.data.TensorDataset(train_images)
train_dataloader = torch.utils.data.DataLoader(train_tensor, batch_size=250, shuffle=True)

In [ ]:
num_epochs = 500
generator.train()
discriminator.train()

for epoch in range(num_epochs):
    gen_loss_avg.append(0)
    disc_loss_avg.append(0)
    num_batches = 0
    
    for image_batch, _ in train_dataloader:
        
        # get dataset image and create real and fake labels for use in the loss
        image_batch = image_batch.to(device)
        label_real = torch.ones(image_batch.size(0), device=device)
        label_fake = torch.zeros(image_batch.size(0), device=device)

        # generate a batch of images from samples of the latent prior
        latent = torch.randn(image_batch.size(0), 100, 1, 1, device=device)
        fake_image_batch = generator(latent)
        
        # train discriminator to correctly classify real and fake
        # (detach the computation graph of the generator and the discriminator,
        # so that gradients are not backpropagated into the generator)
        real_pred = discriminator(image_batch).squeeze()
        fake_pred = discriminator(fake_image_batch.detach()).squeeze()
        disc_loss = 0.5 * (
            F.binary_cross_entropy(real_pred, label_real) +
            F.binary_cross_entropy(fake_pred, label_fake))
        
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()
        
        # train generator to output an image that is classified as real
        fake_pred = discriminator(fake_image_batch).squeeze()
        gen_loss = F.binary_cross_entropy(fake_pred, label_real)
        
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()
        
        gen_loss_avg[-1] += gen_loss.item()
        disc_loss_avg[-1] += disc_loss.item()
        num_batches += 1
        
    gen_loss_avg[-1] /= num_batches
    disc_loss_avg[-1] /= num_batches
    print('Epoch [%d / %d] average loss generator vs. discrim.: %f vs. %f' %
          (epoch+1, num_epochs, gen_loss_avg[-1], disc_loss_avg[-1]))